In [1]:
import pandas as pd
import yfinance as yf

#dfSPY = pd.read_csv("SPY.USUSD_Candlestick_1_D_BID_16.02.2017-21.05.2022.csv")
#dfSPY=yf.download("^GSPC",start='2011-01-05', end='2021-01-05')
dfSPY=yf.download("^RUI",start='2011-01-05', end='2025-04-30')
#dfSPY=yf.download("EURUSD=X",start='2011-01-05', end='2021-01-05')

dfSPY=dfSPY[dfSPY.High!=dfSPY.Low]  
dfSPY.reset_index(inplace=True)
dfSPY.head()

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Price,Date,Close,High,Low,Open,Volume
Ticker,,^RUI,^RUI,^RUI,^RUI,^RUI
0,2011-01-05,706.869995,707.419983,699.049988,704.739990,0
1,2011-01-06,705.479980,707.929993,700.679993,702.380005,0
2,2011-01-07,704.320007,707.929993,698.780029,706.830017,0
3,2011-01-10,703.760010,704.770020,699.020020,703.229980,0
4,2011-01-11,706.520020,707.929993,703.859985,705.090027,0


In [2]:
print(dfSPY.columns)
print(type(dfSPY.columns))

MultiIndex([(  'Date',     ''),
            ( 'Close', '^RUI'),
            (  'High', '^RUI'),
            (   'Low', '^RUI'),
            (  'Open', '^RUI'),
            ('Volume', '^RUI')],
           names=['Price', 'Ticker'])
<class 'pandas.core.indexes.multi.MultiIndex'>


In [3]:
import pandas_ta as ta

In [4]:
# Flatten the MultiIndex columns
dfSPY.columns = [f"{col1}_{col2}" if col2 else col1 for col1, col2 in dfSPY.columns]

# Check new columns
print(dfSPY.columns)

# Now your columns will look like: ['Date', 'Close_^RUI', 'High_^RUI', 'Low_^RUI', 'Open_^RUI', 'Volume_^RUI', 'EMA', 'RSI']

# Rename 'Close_^RUI' to 'Close' for easier indicator calculation
dfSPY.rename(columns={'Close_^RUI': 'Close'}, inplace=True)
dfSPY.rename(columns={'Open_^RUI': 'Open'}, inplace=True)
dfSPY.rename(columns={'High_^RUI': 'High'}, inplace=True)
dfSPY.rename(columns={'Low_^RUI': 'Low'}, inplace=True)
dfSPY.rename(columns={'Volume_^RUI': 'Volume'}, inplace=True)

# Now you can safely apply pandas_ta indicators


dfSPY['EMA'] = ta.sma(dfSPY['Close'], length=200)
dfSPY['RSI'] = ta.rsi(dfSPY['Close'], length=2)
dfSPY.ta.bbands(length=20, std=2.5, append=True)

dfSPY.dropna(inplace=True)
dfSPY.reset_index(drop=True, inplace=True)

dfSPY

Index(['Date', 'Close_^RUI', 'High_^RUI', 'Low_^RUI', 'Open_^RUI',
       'Volume_^RUI'],
      dtype='object')


,Date,Close,High,Low,Open,Volume,EMA,RSI,BBL_20_2.5,BBM_20_2.5,BBU_20_2.5,BBB_20_2.5,BBP_20_2.5
0,2011-10-19,666.960022,679.320007,665.030029,674.260010,0,707.434350,41.559158,595.548746,644.963004,694.377261,15.323129,0.722578
1,2011-10-20,669.909973,672.059998,659.520020,667.630005,0,707.249550,54.360857,597.704805,647.285004,696.865202,15.319434,0.728165
2,2011-10-21,682.700012,683.219971,672.059998,672.059998,0,707.135650,84.259625,598.263168,650.042004,701.820841,15.930920,0.815361
3,2011-10-24,692.719971,693.890015,683.280029,683.280029,0,707.077650,92.232556,596.124520,652.589502,709.054484,17.304901,0.855357
4,2011-10-25,678.590027,691.380005,677.400024,691.380005,0,706.951800,37.977830,595.911512,654.066504,712.221496,17.782593,0.710846
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3395,2025-04-23,2939.669922,2991.389893,2929.300049,2938.739990,0,3144.715560,79.913350,2648.622824,2942.031519,3235.440213,19.945993,0.495976
3396,2025-04-24,3000.550049,3002.699951,2939.389893,2943.659912,0,3144.563861,90.554624,2659.513034,2935.745520,3211.978006,18.818558,0.617301
3397,2025-04-25,3021.540039,3022.949951,2985.429932,3000.729980,0,3144.514462,93.081845,2669.239372,2931.146021,3193.052669,17.870597,0.672569
3398,2025-04-28,3024.310059,3037.709961,2992.040039,3025.050049,0,3144.329312,93.538175,2671.394784,2929.783521,3188.172257,17.638760,0.682915


In [5]:
def addemasignal(df, backcandles):
    emasignal = [0]*len(df)
    for row in range(backcandles, len(df)):
        upt = 1
        dnt = 1
        for i in range(row-backcandles, row+1):
            if df.High[i]>=df.EMA[i]:
                dnt=0
            if df.Low[i]<=df.EMA[i]:
                upt=0
        if upt==1 and dnt==1:
            #print("!!!!! check trend loop !!!!")
            emasignal[row]=3
        elif upt==1:
            emasignal[row]=2
        elif dnt==1:
            emasignal[row]=1
    df['EMASignal'] = emasignal

addemasignal(dfSPY, 6)


In [6]:
def addorderslimit(df, percent):
    ordersignal=[0]*len(df)
    for i in range(1, len(df)): #EMASignal of previous candle!!! modified!!!
        if df.EMASignal[i]==2 and df.Close[i]<=df['BBL_20_2.5'][i]:# and df.RSI[i]<=100: #Added RSI condition to avoid direct close condition
            ordersignal[i]=df.Close[i]-df.Close[i]*percent
        elif df.EMASignal[i]==1 and df.Close[i]>=df['BBU_20_2.5'][i]:# and df.RSI[i]>=0:
            ordersignal[i]=df.Close[i]+df.Close[i]*percent
    df['ordersignal']=ordersignal
    
addorderslimit(dfSPY, 0.00)

In [7]:
dfSPY[dfSPY.ordersignal!=0]

,Date,Close,High,Low,Open,Volume,EMA,RSI,BBL_20_2.5,BBM_20_2.5,BBU_20_2.5,BBB_20_2.5,BBP_20_2.5,EMASignal,ordersignal
118,2012-04-10,750.619995,764.729980,750.030029,763.659973,0,702.808600,0.953716,756.865835,774.960501,793.055167,4.669829,-0.172588,2,750.619995
418,2013-06-20,879.830017,902.469971,877.760010,902.469971,0,833.538099,9.250886,881.826258,905.800000,929.773742,5.293385,-0.041634,2,879.830017
457,2013-08-15,923.619995,937.030029,922.070007,937.030029,0,858.857550,3.584613,925.725225,940.277509,954.829792,3.095317,-0.072333,2,923.619995
458,2013-08-16,920.669983,925.119995,919.150024,923.580017,0,859.567550,2.661574,921.308228,939.367007,957.425787,3.844883,-0.017671,2,920.669983
459,2013-08-19,915.109985,922.460022,915.010010,920.619995,0,860.246350,1.350614,915.740900,938.071005,960.401110,4.760856,-0.014127,2,915.109985
494,2013-10-08,924.559998,937.250000,924.489990,936.729980,0,887.705700,9.961421,925.276037,945.320999,965.365962,4.240880,-0.017861,2,924.559998
568,2014-01-24,999.929993,1021.739990,999.929993,1019.880005,0,946.441652,3.019828,1006.834802,1024.323505,1041.812208,3.414684,-0.197408,2,999.929993
569,2014-01-27,994.500000,1003.020020,989.359985,1000.390015,0,946.992952,2.164061,998.925975,1022.717505,1046.509034,4.652610,-0.093016,2,994.500000
574,2014-02-03,972.820007,997.479980,971.640015,996.570007,0,950.157451,10.459400,973.315595,1014.675000,1056.034405,8.152247,-0.005991,2,972.820007
622,2014-04-11,1013.010010,1024.250000,1012.140015,1019.820007,0,982.207202,13.522048,1014.039786,1040.303998,1066.568210,5.049334,-0.019604,2,1013.010010


# Visualization

In [8]:
import numpy as np
def pointposbreak(x):
    if x['ordersignal']!=0:
        return x['ordersignal']
    else:
        return np.nan
dfSPY['pointposbreak'] = dfSPY.apply(lambda row: pointposbreak(row), axis=1)

In [9]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

dfpl = dfSPY[1000:1250].copy()
#dfpl=dfpl.drop(columns=['level_0'])#!!!!!!!!!!
#dfpl.reset_index(inplace=True)
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),
                go.Scatter(x=dfpl.index, y=dfpl.EMA, line=dict(color='orange', width=2), name="EMA"),
                go.Scatter(x=dfpl.index, y=dfpl['BBL_20_2.5'], line=dict(color='blue', width=1), name="BBL_20_2.5"),
                go.Scatter(x=dfpl.index, y=dfpl['BBU_20_2.5'], line=dict(color='blue', width=1), name="BBU_20_2.5")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointposbreak'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="Signal")

fig.update_layout(
    width=1600,  # make wider
    height=900,  # make taller
    title="Candlestick Chart with EMA and Bollinger Bands",
    xaxis_title="Index",
    yaxis_title="Price"
)
fig.show()

In [10]:
dfpl = dfSPY[:].copy()
def SIGNAL():
    return dfpl.ordersignal

In [13]:
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    initsize = 0.99
    ordertime=[]
    def init(self):
        super().init()
        self.signal = self.I(SIGNAL)

    def next(self):
        super().next()
        
        for j in range(0, len(self.orders)):
            #print('!!!!!!!!!!!!!!!!!!!', self.data.index[-1])
            if self.data.index[-1]-self.ordertime[0]>5:#days max to fulfill the order!!!
                #print('----------------------')
                #print(self.orders)
                #print(self.ordertime)
                self.orders[0].cancel()
                self.ordertime.pop(0)   
            
        if len(self.trades)>0:
            #print(self.data.index[-1], self.trades)
            if self.data.index[-1]-self.trades[-1].entry_time>=10:
                self.trades[-1].close()
                #print(self.data.index[-1], self.trades[-1].entry_time)
            
            if self.trades[-1].is_long and self.data.RSI[-1]>=50:
                self.trades[-1].close()
            elif self.trades[-1].is_short and self.data.RSI[-1]<=50:
                self.trades[-1].close()
        
        if self.signal!=0 and len(self.trades)==0 and self.data.EMASignal==2:
            #Cancel previous orders
            for j in range(0, len(self.orders)):
                self.orders[0].cancel()
                self.ordertime.pop(0)
            #Add new replacement order
            self.buy(sl=self.signal/2, limit=self.signal, size=self.initsize)
            self.ordertime.append(self.data.index[-1])
        
        elif self.signal!=0 and len(self.trades)==0 and self.data.EMASignal==1:
            #Cancel previous orders
            for j in range(0, len(self.orders)):
                self.orders[0].cancel()
                self.ordertime.pop(0)
            #Add new replacement order
            self.sell(sl=self.signal*2, limit=self.signal, size=self.initsize)
            self.ordertime.append(self.data.index[-1])

bt = Backtest(dfpl, MyStrat, cash=10000, margin=1/10, commission=.00)
stat = bt.run()
stat

c:\ProgramData\anaconda3\Lib\site-packages\backtesting\_plotting.py:55: UserWarning:

Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.



Loading BokehJS ...

C:\Users\agtbe\AppData\Local\Temp\ipykernel_26808\897054437.py:52: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



Backtest.run:   0%|          | 0/3399 [00:00<?, ?bar/s]

Start                                     0.0
End                                    3399.0
Duration                               3399.0
Exposure Time [%]                     3.38235
Equity Final [$]                  92655.97925
Equity Peak [$]                   92655.97925
Return [%]                          826.55979
Buy & Hold Return [%]               356.12328
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Alpha [%]                           676.28732
Beta                                  0.42197
Max. Drawdown [%]                   -56.19882
Avg. Drawdown [%]                    -9.32783
Max. Drawdown Duration                  417.0
Avg. Drawdown Duration               82.63636
# Trades                                 30.0
Win Rate [%]                         83.33333
Best Trade [%]                    

In [14]:
bt.plot(show_legend=False)

c:\ProgramData\anaconda3\Lib\site-packages\backtesting\_plotting.py:701: UserWarning:

found multiple competing values for 'toolbar.active_drag' property; using the latest value

c:\ProgramData\anaconda3\Lib\site-packages\backtesting\_plotting.py:701: UserWarning:

found multiple competing values for 'toolbar.active_scroll' property; using the latest value



GridPlot(id='p1337', ...)

In [15]:
stat._trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,SL,TP,PnL,ReturnPct,EntryTime,ExitTime,Duration,Tag,Entry_SIGNAL,Exit_SIGNAL
0,112,419,422,879.820007,879.820007,439.915009,None,0.000000,0.000000,419,422,3,None,0.000000,0.000000
1,107,458,463,923.580017,922.320007,NaN,None,-134.821045,-0.001364,458,463,5,None,920.669983,0.000000
2,105,495,497,924.489990,943.599976,NaN,None,2006.548462,0.020671,495,497,2,None,0.000000,0.000000
3,117,569,573,999.929993,1001.299988,NaN,None,160.289429,0.001370,569,573,4,None,994.500000,0.000000
4,117,623,625,1013.010010,1030.430054,NaN,None,2038.145142,0.017196,623,625,2,None,0.000000,0.000000
5,129,699,701,1074.089966,1077.959961,NaN,None,499.229370,0.003603,699,701,2,None,1072.689941,0.000000
6,133,742,744,1081.380005,1096.089966,NaN,None,1956.424805,0.013603,742,744,2,None,0.000000,0.000000
7,147,793,796,1111.900024,1124.619995,NaN,None,1869.835693,0.011440,793,796,3,None,1104.599976,0.000000
8,159,850,852,1141.060059,1152.540039,NaN,None,1825.316895,0.010061,850,852,2,None,0.000000,0.000000
9,174,927,929,1149.199951,1162.630005,NaN,None,2336.829346,0.011686,927,929,2,None,0.000000,0.000000
